In [5]:
phi = space.set_scale(1)


AttributeError: 'TensorSpace' object has no attribute 'set_scale'

In [ ]:
from pylyra import *




In [ ]:
print()


x = (t, r, th, vph) = sp.symbols('t r theta varphi', real=True)

st = SpaceTime(
    coords=x,
    metric=sp.diag(sp.exp(sp.Function('f1')(r)), -sp.exp(f2 = sp.Function('f2')(r)), -r**2, -r**2*sp.sin(th)**2),
)

# nesse ponto, phi=1, tau=0, M=0


# lembre da classe builtn Tensor. Ela deve receber uma métrica e conexão e escala no cconstrutor. Tensor deve ter informações sobre: nome (id unico), label (expressao para identificar o tensor, R, T, T_i) rank


# st deve ter um atributo interno .connection ou .gamma. Crie uma classe interna para abstrair isso


st.set_scale(sp.Function('phi')(r))

st.scale   #-> isso deve retornar uma instancia de Tensor de rank 0


st.set_torsion(..) # adicona a torção

st.set_non_metricity(...)


st.tensor.from_function(...)
st.tensor.generic(...)
st.tensor.generic(...)


# Algo parecido com isso seria legal


A = st.tensor.from_function( ..., signature=(d,d)) 
B = st.tensor.from_function( ..., signature=(d,d, d)) 


_,a,b,c = st.index("empty a b c")

st.contract(A.idx(up=(a,_),down=(.,a)), indexes=(a,)]  #-> reconhece automaticamente os indices repetidos; o resultado deve ser um tensor de rank 0 

st.contract(A.idx(up=(a,b),down=(_,_)), B.idx(up=(_,_,_),down=(a,b,c))] #-> reconhece automaticamente os indices repetidos; o resultado deve ser um tensor de rank 1


st.eval_contract("A^{a,}_{ ,a}")    #-> reconhece automaticamente os indices repetidos; o resultado deve ser um tensor de rank 0 
st.eval_contract("A^{a,b} B_{a,b,c}") #-> reconhece automaticamente os indices repetidos; o resultado deve ser um tensor de rank 1
            


In [13]:
# _

((Ellipsis, 1), 1)

In [6]:
x = (t, r, th, vph) = sp.symbols('t r theta varphi', real=True)


In [1]:
from pylyra import *

# -- Metrics --------

print("Definindo metrica")
t, r, th, vph = sp.symbols('t r theta varphi', real=True)
x = (t, r, th, vph)
dim = 4

f1 = sp.Function('f1')(r)
f2 = sp.Function('f2')(r)

g = sp.diag(sp.exp(f1), -sp.exp(f2), -r**2, -r**2*sp.sin(th)**2)  # g_{mu nu}

detg = sp.simplify(g.det())
sqrt_minus_detg = sp.sqrt(-detg)

# -- Espaço Tensorial ----------
space = TensorSpace(dim=4, coords=x, metric=g)

# -- Escala de Lyra ----------
print("Definindo escala")
phi = space.set_scale(1)

# -- Tensor de Levi Civita ----------
print("Definindo LeviCivita")
eps = space.from_function(
    lambda mu, nu, rho, sig: sqrt_minus_detg * sp.LeviCivita(mu, nu, rho, sig),
    signature=(d, d, d, d),
)

# -- Christoffel ----------
print("Definindo Christoffel")

def create_christoffel_array():
    dim = g.shape[0]
    g_inv = space.metric_inv
    chris = [[[
        (sp.Rational(1, 2) * sum(
            g_inv[a, d] * (sp.diff(g[d, b], x[c]) +
                           sp.diff(g[d, c], x[b]) -
                           sp.diff(g[b, c], x[d]))
            for d in range(dim)
        ))
        for c in range(dim)]
        for b in range(dim)]
        for a in range(dim)]
    return sp.Array(chris)

chris = create_christoffel_array()

# -- Non-Metricity ----------
print("Definindo Non-Metricity")
M = space.from_function(
    lambda a, b, c: 0,
    signature=(u, d, d),
)
space.set_nonmetricity(M)

# -- Torsion ----------
print("Definindo Torsion")

def torsion_axial(mu):
    if mu == 0:
        return sp.Function('s_t')(r)
    if mu == 1:
        return sp.Function('s_r')(r)
    return 0

s = space.from_function(torsion_axial, signature=(u,))


def torsion_trace(mu):
    if mu == 0:
        return sp.Function('v_t')(r)
    if mu == 1:
        return sp.Function('v_r')(r)
    return 0

v = space.from_function(torsion_trace, signature=(d,))


def torsion_component(m, n, r_):
    return (
        sum(eps(d, d, d, d).comp[m, n, r_, sig] * s(u).comp[sig] for sig in range(dim))
        + sp.Rational(1, 3) * (
            g[m, n] * v(d).comp[r_]
            - g[m, r_] * v(d).comp[n]
        )
    )

tau = space.from_function(torsion_component, signature=(d, d, d))
space.set_torsion(tau)

# -- Connection ----------
print("Definindo Connection")

def connection_element(b, n, l):
    return (
        1 / phi.comp * chris[b, n, l]
        - sp.Rational(1, 2) * M(u, d, d)[b, n, l]
        + 1 / (phi.comp) * (
            sp.KroneckerDelta(b, n) * 1 / phi.comp * sp.diff(phi.comp, x[l])
            - sum(1 / phi.comp * g[n, l] * space.metric_inv[b, s] * sp.diff(phi.comp, x[s]) for s in range(dim))
        )
        + sp.Rational(1, 2) * sum(
            space.metric_inv[m, b] * (
                tau(d, d, d)[l, m, n] - tau(d, d, d)[n, l, m] - tau(d, d, d)[m, l, n]
            )
            for m in range(dim)
        )
    )

Gamma = table(connection_element, dim=dim)
space.set_connection(Gamma)

# -- Curvature ----------
print("Definindo Curvature")

def curvature_element(l, a, m, n):
    return (
        1 / (phi.comp**2) * sp.diff(phi.comp * Gamma[l, a, n], x[m])
        - 1 / (phi.comp**2) * sp.diff(phi.comp * Gamma[l, a, m], x[n])
        + sum(Gamma[r, a, n] * Gamma[l, r, m] for r in range(dim))
        - sum(Gamma[r, a, m] * Gamma[l, r, n] for r in range(dim))
    )

Curv = space.from_function(curvature_element, signature=(u, d, d, d))

# -- Ricci ----------
print("Definindo Ricci")

def ricci_element(a, m):
    return sp.expand(sp.simplify(
        sum(Curv(u, d, d, d).comp[l, a, m, l] for l in range(dim))
    ))

Ricc = space.from_function(ricci_element, signature=(d, d))

# -- Curvature Scalar ----------
print("Definindo Curvature Scalar")
R = sp.expand(sum(Ricc(u, d).comp[l, l] for l in range(dim))).subs(th, sp.pi/2)


Definindo metrica
Definindo escala
Definindo LeviCivita
Definindo Christoffel
Definindo Non-Metricity
Definindo Torsion
Definindo Connection
Definindo Curvature
Definindo Ricci
Definindo Curvature Scalar


In [2]:
from pylyra import *
# supondo que space já tenha coords e connection (Gamma) definidos
T = space.from_function(lambda a, b: sp.Function("T_%s%s" % (a,b))(*space.coords),
                        signature=(u, d))

# adiciona o índice covariante no fim: (u, d, d)
dT = T.nabla(deriv_position="append")

# componente: dT[a,b,k] = ∇_k T^a_b
dT.comp[0, 1, 2]


r**2*T_21(t, r, theta, varphi)*v_t(r)*exp(-f1(r))/3 + r**2*T_31(t, r, theta, varphi)*s_r(r)*sqrt(exp(f1(r) + f2(r)))*exp(-f1(r))*Abs(sin(theta))/2 - T_02(t, r, theta, varphi)*v_r(r)/3 - T_02(t, r, theta, varphi)*Derivative(phi(r), r)/phi(r)**2 + T_03(t, r, theta, varphi)*s_t(r)*sqrt(exp(f1(r) + f2(r)))*Abs(sin(theta))/(2*sin(theta)**2) + Derivative(T_01(t, r, theta, varphi), theta) - T_02(t, r, theta, varphi)/(r*phi(r))

Na verdade, queria melhorar as abstrações. Da forma como está agora:

- A métrica não é um tensor
- Escalares não são tensores
- Ter alguma forma de fazer contrações


Pensei em ter um elemento que abstraia o espaço-tempo. Manifold? Não sei

Dai, deveriamos ter métodos set_scale. Se não passar, usa-se phi = sp.Function('phi')(r)

um método set metric
set torsion
set metrica compatibility

e uma forma de fazer contração de indices


In [ ]:
M.contract(
    
)

In [1]:
from pylyra import *

t, r, th, vph = sp.symbols('t r theta varphi', real=True)
x = (t, r, th, vph)

st = SpaceTime(
    dim=4,
    coords=x,
    metric=sp.diag(
        sp.exp(sp.Function('f1')(r)),
        -sp.exp(sp.Function('f2')(r)),
        -r**2,
        -r**2*sp.sin(th)**2
    ),
)

# defaults
st.scale      # Tensor rank 0 (phi = 1)
st.torsion    # tau = 0
st.nonmetricity  # M = 0

st.set_scale(sp.Function('phi')(r))

A = st.tensor.from_function(lambda i, j: sp.Function('A')(r), signature=(d, d), name="A")
B = st.tensor.from_function(lambda i, j, k: sp.Function('B')(r), signature=(d, d, d), name="B")

_, a, b, c = st.index("empty a b c")

# índices por posição (up/down por posição do rank)
res1 = st.contract(A.idx(up=(a, _), down=(_, a)))

# contração entre dois tensores
res2 = st.contract(
    A.idx(up=(a, b), down=(_, _)),
    B.idx(up=(_, _, _), down=(a, b, c))
)

# # parsing por string
# res3 = st.eval_contract("A^{a,}_{ ,a}")
# res4 = st.eval_contract("A^{a,b} B_{a,b,c}")


In [2]:
st.scale

AttributeError: 'ImmutableDenseNDimArray' object has no attribute '_repr_html_'